## Классификация текстов: Spam or Ham

В этом задании на примере классического датасета Spambase Dataset (https://archive.ics.uci.edu/ml/datasets/spambase) мы попробуем сделать свой спам-фильтр c помощью библиотеки scikit-learn. Датасет содержит корпус текстов 5,574 смс с метками "spam" и "ham".

### Данные

Для удобства данные приложены в описании задания

In [2]:
import pandas as pd
df = pd.read_csv('3_data.csv', encoding='latin-1')

Оставляем только интересующие нас колонки — тексты смс и метки:

In [3]:
df = df[['v1', 'v2']]
df = df.rename(columns = {'v1': 'label', 'v2': 'text'})
df.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Удаляем дублирующиеся тексты:

In [4]:
df = df.drop_duplicates('text')

Заменяем метки на бинарные:

In [5]:
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

### Предобработка текста (Задание 1)

Нужно дополнить функцию для предобработки сообщений, которая делает с текстом следующее:
* приводит текст к нижнему регистру;
* удаляет стоп-слова;
* удаляет знаки препинания;
* нормализует текст при помощи стеммера Snowball.

Предлагаем воспользоваться библиотекой nltk, чтобы не составлять список стоп-слов и не реализовывать алгоритм стемминга самостоятельно. Примеры использования стеммеров можно найти по ссылке (https://www.nltk.org/howto/stem.html).

In [9]:
from nltk import stem
from nltk.corpus import stopwords
import re
import nltk

nltk.download("stopwords")

stemmer = stem.SnowballStemmer('english')
STOPWORDS = set(stopwords.words('english'))

def preprocess(text):
    # Removing non-alphabetical characters and converting text to lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()

    # Tokenization (splitting text into words)
    words = text.split()

    # Removing stopwords and applying stemming
    filtered_words = [stemmer.stem(word) for word in words if word not in STOPWORDS]

    # Joining words back to form the processed text
    return ' '.join(filtered_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Проверка, что функция работает верно

In [10]:
assert preprocess("I'm gonna be home soon and i don't want to talk about this stuff anymore tonight, k? I've cried enough today.") == "im gonna home soon dont want talk stuff anymor tonight k ive cri enough today"
assert preprocess("Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...") == "go jurong point crazi avail bugi n great world la e buffet cine got amor wat"

Применяем получившуюся функцию к текстам:

In [11]:
df['text'] = df['text'].apply(preprocess)

### Разделение данных на обучающую и тестовую выборки (Задание 2)

In [12]:
y = df['label'].values

Теперь нужно разделить данные на тестовую (test) и обучающую (train) выборку. В библиотеке scikit-learn для этого есть готовые инструменты.

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['text'], y, test_size=0.3, random_state=51)

### Обучение классификатора (Задание 3)

Переходим к обучению классификатора.

Сначала извлекаем признаки из текстов. Рекомендуем попробовать разные способы и посмотреть, как это влияет на результат (подробнее о различных способах представления текстов можно прочитать по ссылке https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction).

Затем обучаем классификатор. Мы используем SVM, но можно поэкспериментировать с различными алгоритмами.

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# извлекаем признаки из текстов
vectorizer = TfidfVectorizer(decode_error='ignore')
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [15]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

#обучаем подель SVM

model = LinearSVC(random_state = 51, C = 1)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

Для самопроверки. Если вы верно дополнили функцию ```preprocess```, то должны получиться следующие результаты оценки модели.

In [ ]:
print(classification_report(y_test, predictions, digits=3))

              precision    recall  f1-score   support

           0      0.984     0.993     0.988      1355
           1      0.946     0.888     0.916       196

    accuracy                          0.979      1551
   macro avg      0.965     0.940     0.952      1551
weighted avg      0.979     0.979     0.979      1551



In [16]:
print(classification_report(y_test, predictions, digits=3))

              precision    recall  f1-score   support

           0      0.984     0.992     0.988      1355
           1      0.941     0.888     0.913       196

    accuracy                          0.979      1551
   macro avg      0.962     0.940     0.951      1551
weighted avg      0.978     0.979     0.978      1551



Выполним предсказание для конкретного текста

In [17]:
txt = "As a valued customer, I am pleased to advise you that following recent review of your Mob No. you are awarded with a å£1500 Bonus Prize, call 09066364589"
txt = preprocess(txt)
txt = vectorizer.transform([txt])

In [18]:
model.predict(txt)

array([1])

In [ ]:
Сообщение помечено как спам.

#  Задание Блок 3

Используя в качестве исходного набора данных датасет Spambase Dataset (тот же, что и в примере), постройте модель для определения является ли сообщение спамом.

Выполните предварительную обработку текста (как в примере) и разделите датасет на тренировочный и тестовый набор данных с параметрами test_size = 0.3, random_state = 17. Обучите классификатор SVM при C = 1.2 и random_state = 17 на обучающей выборке и произведите оценку полученной модели на тестовой.

Оцените модель на тестовых данных.

In [19]:
import pandas as pd
df = pd.read_csv('3_data.csv', encoding='latin-1')

df = df[['v1', 'v2']]
df = df.rename(columns = {'v1': 'label', 'v2': 'text'})

df = df.drop_duplicates('text')

df['label'] = df['label'].map({'ham': 0, 'spam': 1})



In [20]:
from nltk import stem
from nltk.corpus import stopwords
import re
import nltk

nltk.download("stopwords")

stemmer = stem.SnowballStemmer('english')
STOPWORDS = set(stopwords.words('english'))

def preprocess(text):
    # Removing non-alphabetical characters and converting text to lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()

    # Tokenization (splitting text into words)
    words = text.split()

    # Removing stopwords and applying stemming
    filtered_words = [stemmer.stem(word) for word in words if word not in STOPWORDS]

    # Joining words back to form the processed text
    return ' '.join(filtered_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
df['text'] = df['text'].apply(preprocess)

In [22]:
y = df['label'].values

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['text'], y, test_size=0.3, random_state=17)

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# извлекаем признаки из текстов
vectorizer = TfidfVectorizer(decode_error='ignore')
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [25]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

#обучаем подель SVM

model = LinearSVC(random_state = 17, C = 1.2)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

In [27]:
print(classification_report(y_test, predictions, digits=4))

              precision    recall  f1-score   support

           0     0.9841    0.9985    0.9913      1367
           1     0.9878    0.8804    0.9310       184

    accuracy                         0.9845      1551
   macro avg     0.9860    0.9395    0.9612      1551
weighted avg     0.9846    0.9845    0.9841      1551



Screw this self-help industry crap.

V-E-R-I-Z-O-N, To continue using our services please go to update.vtext02.net and update your account.359394.

Hello, I have a special offer for you… WANT TO LOSE WEIGHT? The most powerful weightloss is now available without prescription. All natural Adipren720 100% Money Back Guarantee!

This is a fine collection of articles by Bernard Lewis about the Middle East.

In [28]:
txt_lst = ["Screw this self-help industry crap.",
           "V-E-R-I-Z-O-N, To continue using our services please go to update.vtext02.net and update your account.359394.",
           "Hello, I have a special offer for you… WANT TO LOSE WEIGHT? The most powerful weightloss is now available without prescription. All natural Adipren720 100% Money Back Guarantee!",
           "This is a fine collection of articles by Bernard Lewis about the Middle East."]

for i, txt in enumerate(txt_lst):
  txt_temp = preprocess(txt)
  txt_temp = vectorizer.transform([txt_temp])
  temp_res = model.predict(txt_temp)
  print(f"Text {i}: prediction={temp_res}")

Text 0: prediction=[0]
Text 1: prediction=[1]
Text 2: prediction=[0]
Text 3: prediction=[0]
